# Demo đơn giản với RNN (Recurrent Neural Network)

Notebook này minh họa cách dùng **RNN** để giải một bài toán rất đơn giản: **phân loại chuỗi nhị phân theo tính chẵn lẻ (parity)**.

Bài toán:
- Input: một chuỗi gồm 0 và 1, ví dụ: `[1, 0, 1, 1, 0, 0, 1, 0]`
- Output: nhãn 0 nếu tổng số bit 1 là **chẵn**, nhãn 1 nếu là **lẻ**.

RNN sẽ đọc từng phần tử của chuỗi theo thứ tự và học cách "nhớ" thông tin để quyết định nhãn cuối.

## 1. RNN là gì (tóm tắt nhanh)?

RNN (Recurrent Neural Network) là mạng nơ-ron **tuần tự**, phù hợp với dữ liệu dạng chuỗi (sequence):
- Ví dụ: câu chữ, chuỗi thời gian, tín hiệu cảm biến...
- Ở mỗi thời điểm $t$, RNN nhận input $x_t$ và hidden state $h_{t-1}$, rồi sinh ra $h_t$.

Công thức đơn giản:
$$h_t = f(W_x x_t + W_h h_{t-1} + b)$$

Trong demo này, ta dùng **`nn.RNN` của PyTorch** để xử lý chuỗi nhị phân.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
import numpy as np

torch.manual_seed(42)
np.random.seed(42)
device = 'cpu'
device

## 2. Tạo dữ liệu: chuỗi nhị phân & nhãn chẵn/lẻ

Ta sinh ngẫu nhiên nhiều chuỗi nhị phân có độ dài cố định (ví dụ 8):
- Nếu số lượng bit 1 trong chuỗi là **chẵn** → nhãn 0
- Nếu là **lẻ** → nhãn 1

In [ ]:
def generate_binary_sequences(n_samples=1000, seq_len=8):
    # Sinh ngẫu nhiên 0/1
    X = np.random.randint(0, 2, size=(n_samples, seq_len))  # shape: (N, seq_len)
    # Nhãn parity: tổng bit 1 mod 2
    y = X.sum(axis=1) % 2  # 0: chẵn, 1: lẻ
    return X.astype(np.float32), y.astype(np.int64)

seq_len = 8
X, y = generate_binary_sequences(n_samples=2000, seq_len=seq_len)
print('X shape:', X.shape)
print('y shape:', y.shape)
print('Ví dụ một chuỗi:', X[0], '→ nhãn:', y[0])

## 3. Chia train/test và chuyển sang Tensor PyTorch

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Định dạng cho RNN với batch_first=True: (batch_size, seq_len, input_size)
X_train_tensor = torch.from_numpy(X_train).unsqueeze(-1)  # input_size = 1
X_test_tensor = torch.from_numpy(X_test).unsqueeze(-1)
y_train_tensor = torch.from_numpy(y_train)
y_test_tensor = torch.from_numpy(y_test)

print('X_train_tensor shape:', X_train_tensor.shape)
print('y_train_tensor shape:', y_train_tensor.shape)

## 4. Định nghĩa mô hình RNN đơn giản

Cấu trúc mô hình:
- Một lớp `nn.RNN` với `input_size = 1`, `hidden_size = 16`, `batch_first=True`.
- Lấy hidden state cuối cùng `h_last` → đưa qua `nn.Linear(hidden_size, 2)` để phân loại 2 lớp (chẵn/lẻ).

In [ ]:
class SimpleRNNParity(nn.Module):
    def __init__(self, input_size=1, hidden_size=16, num_layers=1, num_classes=2):
        super().__init__()
        self.rnn = nn.RNN(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True
        )
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # x: (batch, seq_len, input_size)
        out, h_n = self.rnn(x)  # out: (batch, seq_len, hidden_size), h_n: (num_layers, batch, hidden_size)
        # Lấy hidden state cuối cùng của bước thời gian cuối cùng
        last_hidden = out[:, -1, :]  # (batch, hidden_size)
        logits = self.fc(last_hidden)  # (batch, num_classes)
        return logits

model = SimpleRNNParity(input_size=1, hidden_size=16, num_layers=1, num_classes=2).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

model

## 5. Huấn luyện mô hình RNN

Ta huấn luyện mô hình trong ~30 epochs, batch_size đơn giản toàn bộ (full batch) để code ngắn gọn.

In [ ]:
num_epochs = 30

X_train_tensor = X_train_tensor.to(device)
y_train_tensor = y_train_tensor.to(device)

for epoch in range(1, num_epochs + 1):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    if epoch % 5 == 0 or epoch == 1:
        # Tính accuracy trên tập train
        _, predicted = torch.max(outputs, dim=1)
        train_acc = (predicted == y_train_tensor).float().mean().item()
        print(f"Epoch [{epoch}/{num_epochs}] - Loss: {loss.item():.4f} - Train Acc: {train_acc:.4f}")

## 6. Đánh giá mô hình trên tập test

In [ ]:
model.eval()
with torch.no_grad():
    X_test_tensor = X_test_tensor.to(device)
    y_test_tensor = y_test_tensor.to(device)
    outputs_test = model(X_test_tensor)
    _, predicted_test = torch.max(outputs_test, dim=1)
    test_acc = (predicted_test == y_test_tensor).float().mean().item()

print('Test accuracy:', test_acc)

## 7. Thử dự đoán với vài chuỗi cụ thể

Ta tạo một vài chuỗi cụ thể và xem RNN dự đoán như thế nào.

In [ ]:
def predict_sequence(seq_list):
    model.eval()
    arr = np.array(seq_list, dtype=np.float32).reshape(1, -1, 1)  # (1, seq_len, 1)
    x_tensor = torch.from_numpy(arr).to(device)
    with torch.no_grad():
        logits = model(x_tensor)
        probs = torch.softmax(logits, dim=1)
        pred_class = torch.argmax(probs, dim=1).item()
    return pred_class, probs.cpu().numpy()[0]

test_sequences = [
    [0, 0, 0, 0, 0, 0, 0, 0],  # 0 bit 1 → chẵn → nhãn 0
    [1, 1, 0, 0, 0, 0, 0, 0],  # 2 bit 1 → chẵn → nhãn 0
    [1, 0, 0, 0, 0, 0, 0, 0],  # 1 bit 1 → lẻ → nhãn 1
    [1, 0, 1, 0, 1, 0, 1, 0],  # 4 bit 1 → chẵn → nhãn 0
]

for seq in test_sequences:
    pred, probs = predict_sequence(seq)
    true_label = sum(seq) % 2
    print(f"Chuỗi: {seq} | Tổng bit 1 = {sum(seq)} -> Nhãn thật: {true_label} | Dự đoán: {pred} | Probs: {probs}")

## 8. Tóm tắt

Trong demo này, bạn đã thấy:
- Cách sinh dữ liệu chuỗi nhị phân đơn giản cho bài toán parity.
- Cách xây dựng một mô hình **RNN đơn giản** bằng PyTorch với `nn.RNN`.
- Cách huấn luyện mô hình để phân loại chuỗi (many-to-one).
- Cách kiểm tra độ chính xác và thử dự đoán với vài chuỗi cụ thể.

Bạn có thể mở rộng bài toán này sang:
- Dùng `nn.LSTM` hoặc `nn.GRU` thay cho `nn.RNN`.
- Thay đổi độ dài chuỗi, kiểu dữ liệu, hoặc bài toán (ví dụ dự đoán ký tự tiếp theo, bài toán cộng dồn, v.v.).